### Task 1.1


In [9]:
def extract_words_tokens(any_string: str):
    without_space_string = any_string.strip().split()
    num_words = len(without_space_string)
    num_tokens = len(any_string)
    return(print(any_string, ":", "num_words:", num_words, "and", "num_tokens:", num_tokens, "respectively"))

### Task 1.2

In [10]:
def lemmatize(any_string, file_name):
    lemma_dict = {}
    with open(file_name, 'r', encoding="utf-8-sig") as file:
        for line in file:
            lemma, word_form = line.strip().split("\t")
            lemma_dict[word_form] = lemma
    
    input_words = any_string.split()
    
    dictionary_of_lemmatized_words = {}
    for word in input_words:
        word_without_special_characters = ''.join(e for e in word if e.isalnum())
        lower_word_without_special_characters = word_without_special_characters.lower()

        dictionary_of_lemmatized_words[word_without_special_characters] = lemma_dict.get(lower_word_without_special_characters, lower_word_without_special_characters)
    
    return dictionary_of_lemmatized_words

### Task 2

### Task 2.1

### Task 3

In [11]:
import csv
from collections import Counter

def create_count_and_probability(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()
    
    vocabulary = set()
    for line in lines:
        lemmatized_words = lemmatize(line, "lemmatization-en.txt").values()
        vocabulary.update(lemmatized_words)

    vocabulary = sorted(vocabulary)
    
    with open('output.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Text', 'Count_Vector', 'Probability']) 
        
        for line in lines:
            lemmatized_words_dict = lemmatize(line, "lemmatization-en.txt")

            lemmatized_words = []
            for word in line.split():
                word_without_special_characters = ''.join(e for e in word if e.isalnum()).lower()
                lemmatized_word = lemmatized_words_dict.get(word_without_special_characters, word_without_special_characters)
                lemmatized_words.append(lemmatized_word)
            
            word_count = Counter(lemmatized_words)
            
            count_vector = [word_count[word] for word in vocabulary]

            total_words = sum(count_vector)
            probability_vector = [word_count[word] / total_words if total_words > 0 else 0 for word in lemmatized_words]
            writer.writerow([line.strip(), count_vector, probability_vector])

    return ('output.csv')

create_count_and_probability("corpus.txt")


'output.csv'